In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets
from IPython.display import display, clear_output
import scipy.signal as signal
import time
import warnings
warnings.filterwarnings('ignore')

# Sampling frequency
Fs = 8192  
# Signal duration
t = np.arange(0, 1.0, 1/Fs)  
# Re-define the signal with new frequencies
s_new = np.sin(2*np.pi*400*t) + np.sin(2*np.pi*950*t) + np.sin(2*np.pi*1500*t) + np.sin(2*np.pi*3000*t)

# Re-define the filters with different lengths
filters5 = {
    'Equiripple Filter 32+1 Q3': signal.remez(33, [0, 0.1*Fs, 0.15*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 32+1 Q4': signal.remez(33, [0, 0.11*Fs, 0.12*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 64+1 Q3': signal.remez(65, [0, 0.1*Fs, 0.15*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 64+1 Q4': signal.remez(65, [0, 0.11*Fs, 0.12*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 128+1 Q3': signal.remez(129, [0, 0.1*Fs, 0.15*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 128+1 Q4': signal.remez(129, [0, 0.11*Fs, 0.12*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 140+1 Q3': signal.remez(141, [0, 0.1*Fs, 0.15*Fs, 0.5*Fs], [1, 0], Hz=Fs),
    'Equiripple Filter 140+1 Q4': signal.remez(141, [0, 0.11*Fs, 0.12*Fs, 0.5*Fs], [1, 0], Hz=Fs),
}

# Create checkboxes for each filter
checkboxes5 = [widgets.Checkbox(value=True, description=name) for name in filters5.keys()]

# Create output widget
output_plot5 = widgets.Output()

# Update plot function
def update_plot5(dummy=None):
    with output_plot5:
        clear_output(wait=True)
        # Start a new plot
        plt.figure(figsize=(14, 4))
        # Add original signal PSD
        f, Pxx_den_original = signal.welch(s_new, Fs, nperseg=1024)
        plt.semilogy(f, Pxx_den_original, label='Original Signal')
        # Plot PSD for each selected filter
        for cb in checkboxes5:
            if cb.value:  # only plot if the checkbox is checked
                filter_name = cb.description
                filter_coeffs = filters5[filter_name]
                filtered_signal = signal.lfilter(filter_coeffs, 1.0, s_new)
                f, Pxx_den_filtered = signal.welch(filtered_signal, Fs, nperseg=1024)
                plt.semilogy(f, Pxx_den_filtered, label=filter_name)
        # Formatting the plot
        plt.title('Power Spectral Density')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power/Frequency (dB/Hz)')
        plt.legend()
        plt.grid(which='both', axis='both')
        plt.show()

# Attach the update_plot function to the checkboxes
for cb in checkboxes5:
    cb.observe(update_plot5, names='value')

update_plot5()

# HTML Label for enhanced UI
html_label = widgets.HTML(
    value="""
    <h2 style='font-weight: bold; font-size: 30px; text-align: center; margin: 20px 0 0 0;'>Power Spectral Density</h2>
    <h4 style='font-weight: bold; font-size: 16px; text-align: center; margin: 0 0 20px 0;'>(using Parks-McClellan filters)</h4>
    """
)

vbox_checkboxes5 = widgets.VBox(checkboxes5)

ui = widgets.HBox([vbox_checkboxes5], layout=Layout(justify_content='center', align_items='center'))

out = widgets.VBox([html_label, ui, output_plot5])

# Display the checkboxes and the plot
display(out)

# Initialize the first plot update
update_plot5()